In [3]:
from Bio import SeqIO

In [6]:
# parsing the whole sequence
records = list(SeqIO.parse(".\data\gisaid_hcov-19_2020_05_10_19.fasta", "fasta"))
record_counter = 0
file_count = 0
no_n_count = 0
file = list()
partition = list()
dict_seq = {}
for record in records:
    # correcting the sequence id by replacing space ' ' with underscore '_'
    temp = record.description
    temp = temp.replace(" ", "_")
    record.description = temp
    record.id = temp
    
    # taking a rough range that contains spike protein
    record_counter += 1
    record.seq = record.seq[21000:26900]
    should_remove = 0;
    
    normal = ['A', 'T', 'G', 'C']
    
    record.seq = record.seq.upper()
    
    # removing sequences that have special characters
    for k in record.seq:
        if k not in normal:
            should_remove = 1
    
    if should_remove == 0:
        file.append(record)
        partition.append(record)
        no_n_count += 1
    
#     if record_counter % 100 == 0:
#         SeqIO.write(partition, ".\data\spike_" + str(file_count) + ".fasta", "fasta")
#         file_count += 1
#         partition.clear()
    
SeqIO.write(file,".\data\gisaid_hcov-19_spike_protein_no_special_char.fasta","fasta")
print(record_counter)
print(no_n_count)

12956
10446


In [7]:
# reading sequences that have no special characters
sequences = list(SeqIO.parse(".\data\gisaid_hcov-19_spike_protein_no_special_char.fasta", "fasta"))

# using translation table 1
table = 1

# defining the length of nt
# @param exact_len is the length of Spike protein amino acid
# @param stop is the length of Spike protein amino acid + one stop codon
min_pro_len = 1200
exact_len = 1273
stop = 1274
max_pro_len = 1276

# @param count is the count of sequences that have been processed
# @param count_count is the count of sequences that have exactly 1273 amino acids
count = 0
count_count = 0

specific_spike = list()
need_to_improve = list()

for seq in sequences:
    count += 1
    seq_spike = seq.seq
    for k in range(1500):
        codon = seq_spike[k:k+3]
        length = 3 * ((len(seq_spike)-k) // 3)
        # looking for the start codon
        if codon == 'ATG' or codon == 'atg':
            # translating
            for pro in seq_spike[k:length].translate(table).split("*"):
                pro_len = len(pro)
                # if the length of the translated protein is 1273 +/- 3
                if pro_len >= exact_len - 3 and pro_len <= exact_len + 3:
                    seq.seq = seq_spike[k:stop*3+k]
                    count_count += 1
                    break
                # if the length of the translated protein is too long
                elif pro_len>min_pro_len:
                    seq.seq = seq_spike[k:stop*3+k]
                    
    if len(seq.seq) == 3822:
        specific_spike.append(seq)
    else:
        need_to_improve.append(seq)

SeqIO.write(specific_spike, ".\data\gisaid-hcov-19_spike_protein_no_spe.fasta", "fasta")
SeqIO.write(need_to_improve, ".\data\gisaid-hcov-19_spike_protein_no_spe_too_long.fasta", "fasta")
print("total: " + str(count))
print(str(count_count))
print("------------------")

total: 10446
10446
------------------


In [8]:
# a dictionary for unique sequences
# key is the sequence
# value is a fasta object
spike_dict = {}
spike = list(SeqIO.parse(".\data\gisaid-hcov-19_spike_protein_no_spe.fasta", "fasta"))

for sp in spike:
    spike_dict[sp.seq] = sp
print(len(spike_dict))

781


c:\softwares\python\lib\site-packages\Bio\Seq.py:165: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


In [13]:
# a list of unique sequences
unique = list()
for sp in spike_dict.keys():
    unique.append(spike_dict[sp])
print(str(len(unique)))
print(unique)

SeqIO.write(unique, r".\data\unique_training.fasta","fasta")

781
[SeqRecord(seq=Seq('ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTT...TAA', SingleLetterAlphabet()), id='hCoV-19/Wales/PHWC-2A167/2020|EPI_ISL_432184|2020-04-05', name='hCoV-19/Wales/PHWC-2A167/2020|EPI_ISL_432184|2020-04-05', description='hCoV-19/Wales/PHWC-2A167/2020|EPI_ISL_432184|2020-04-05', dbxrefs=[]), SeqRecord(seq=Seq('ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTT...ACG', SingleLetterAlphabet()), id='hCoV-19/India/1-27/2020|EPI_ISL_413522|2020-01-27', name='hCoV-19/India/1-27/2020|EPI_ISL_413522|2020-01-27', description='hCoV-19/India/1-27/2020|EPI_ISL_413522|2020-01-27', dbxrefs=[]), SeqRecord(seq=Seq('ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTT...TAA', SingleLetterAlphabet()), id='hCoV-19/India/1-31/2020|EPI_ISL_413523|2020-01-31', name='hCoV-19/India/1-31/2020|EPI_ISL_413523|2020-01-31', description='hCoV-19/India/1-31/2020|EPI_ISL_413523|2020-01-31', dbxrefs=[]), SeqRecord(seq=Seq('ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTGTTAATCTT...TAA'

781